In [62]:
import pandas as pd

In [77]:
%%time
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the repeatmasker
driver.get("https://www.repeatmasker.org/cgi-bin/AnnotationRequest")
print(driver.title)

# select Hg38
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "genome"))
    )
    dropdown = Select(element)
    dropdown.select_by_visible_text("Human - Dec 2013 - hg38")
except:
    print("can't select genome")


# set flank coords
coords = "chr3:81813044-81813113"
try:
    range = driver.find_element(By.NAME, "range")
    range.clear()
    range.send_keys(coords)
except:
    print("can't pass coords")

# submit
try:
    button = driver.find_element(By.NAME, ".submit")
    button.click()
    print('submitted')
except:
    print("can't submit")

# check results 
res = ''
try:
    element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, "pre")) 
        )
    res = element.text
except:
    print('exception 1')
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/form/font/text()")) 
        )
        print(element.text)
    except:
        print("exception 2")
driver.quit()

try:
    list = res.split('\n')[2].split(' ')
    list = ' '.join(list).split()
    print(f"flank coord: {coords}")
    print(f"repeat coord: {list[4]}:{list[5]}-{list[6]}")
    print("strand:", list[8])
    print("repeat name:", list[9])
    print("repeat type", list[10])
except:
    print('res list error')

Repeat Annotation Request Form
submitted
flank coord: chr3:81813044-81813113
repeat coord: chr3:81813104-81813389
strand: +
repeat name: AluYa5
repeat type SINE/Alu
CPU times: user 27.8 ms, sys: 4 ms, total: 31.8 ms
Wall time: 9.27 s
